<a href="https://colab.research.google.com/github/leonardoleandrodev/covidMachineLearningPrediction/blob/main/DEEP_LEARNING_KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import loadtxt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from pandas import read_csv as rc,factorize as fc,to_numeric as tnum,set_option as op

In [ ]:
# dados do site https://opendatasus.saude.gov.br/dataset/bd-srag-2021
urlfile = 'https://opendatasus.saude.gov.br/dataset/9f76e80f-a2f1-4662-9e37-71084eae23e3/resource/42bd5e0e-d61a-4359-942e-ebc83391a137/download/influd21-18-06-2021.csv'
#https://opendatasus.saude.gov.br/dataset/9f76e80f-a2f1-4662-9e37-71084eae23e3/resource/b3321e55-24e9-49ab-8651-29cf5c8f3179/download/dicionario_de_dados_srag_hospitalizado_23.03.2021.pdf
dataset = rc(urlfile, delimiter=';',encoding='ISO-8859-1')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,18,60,62,63,64,92,94,106,108,115,117,118,119,123,144,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
op('display.max_columns', None)
dataset.head(10)


In [ ]:
amostra = dataset[
                  ['CS_SEXO','NU_IDADE_N','FEBRE','TOSSE',
                   'GARGANTA','DIARREIA','VOMITO','OUTRO_SIN',
                   'CARDIOPATI','HEMATOLOGI','HEPATICA','ASMA',
                   'DIABETES','PNEUMOPATI','RENAL','OBESIDADE',
                   'ANTIVIRAL','UTI','DOR_ABD','FADIGA'
                   ,'PERD_OLFT','PERD_PALA','EVOLUCAO','CLASSI_FIN']]#[1:10000]
amostra = amostra.dropna()
#amostra['IdadeNaDataNotificacao'] = [int(idade.split(' ')[0]) for idade in amostra['IdadeNaDataNotificacao']]
#amostra['IdadeNaDataNotificacao'] = [int(idade.split(' ')[0]) for idade in amostra['IdadeNaDataNotificacao']]
#amostra = amostra.drop(amostra[amostra.ResultadoTesteRapido == 'Não Informado'].index)
#amostra = amostra.drop(amostra[amostra.ResultadoTesteRapido == 'Inconclusivo'].index)
#amostra = amostra.drop(amostra[amostra.ComorbidadePulmao == '-'].index)
#amostra = amostra.drop(amostra[amostra.RacaCor == 'Ignorado'].index)
amostra = amostra.drop(amostra[amostra.NU_IDADE_N < 20 ].index)
amostra = amostra.drop(amostra[amostra.EVOLUCAO > 2 ].index)
amostra = amostra.drop(amostra[amostra.FEBRE > 2 ].index)
amostra = amostra.drop(amostra[amostra.TOSSE > 2 ].index)
amostra = amostra.drop(amostra[amostra.GARGANTA > 2 ].index)
amostra = amostra.drop(amostra[amostra.FADIGA > 2 ].index)
amostra = amostra.drop(amostra[amostra.PERD_OLFT > 2 ].index)
amostra = amostra.drop(amostra[amostra.PERD_PALA > 2 ].index)
amostra = amostra.drop(amostra[amostra.UTI > 2 ].index)
#amostra = amostra.drop(amostra[amostra.Classificacao == 'Suspeito' ].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == 'Ignorado'].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == '-'].index)
#amostra = amostra.drop(amostra[amostra.Evolucao == 'Óbito por outras causas'].index)

amostra.head()


In [ ]:
colunas = ['FADIGA','PERD_OLFT','PERD_PALA','CLASSI_FIN','UTI']
classe = colunas[-1]
amostrax = amostra[colunas]

amostrax['CLASSI_FIN'] = amostrax['CLASSI_FIN']#.apply(lambda x: 1 if x == 5.0 else 0)
#amostrax['CS_SEXO'] = amostrax['CS_SEXO'].replace('F',0).replace('M',1)
#amostrax = amostrax[[c for c in colunas]]
#amostrax['NU_IDADE_N'] = [round(idd/10)*10 for idd  in amostrax['NU_IDADE_N']]
dados = amostrax.replace(1.0,1).replace(2.0,0)

#dados[colunas] = dados[colunas].astype(int)
del dados[classe]
resnegposi = dados[colunas[:-1]].sum(axis=1).apply(lambda x: 1 if x > 6.0 else 0)
#amostrax[[classe]].replace(1.0,1).replace(2.0,0)#.replace('Descartados',0).replace('Confirmados',1).replace('Negativo',0).replace('Positivo',1)
dados[20:].head(10) 
resnegposi.head()

In [ ]:

dadosX = dados.apply(lambda col: fc(col)[0] if col.name == 'CS_SEXO' else col)
dadosX = dadosX.apply(tnum)
resnegposiY = resnegposi#.apply(lambda col: fc(col)[0])

#resnegposiY.info()

In [ ]:
from sklearn.model_selection import train_test_split
val_treino,val_teste,val_res_treino,val_res_teste = train_test_split(dadosX,resnegposiY,test_size=0.001,train_size=0.01)
(len(val_treino),len(val_teste))

In [ ]:
# definir o modelo com keras
# inicializar o modelo sequencial
model = Sequential()
# inicializar a primeira camada, com 12 neurÃ´nios, 8 entradas utilizando a funÃ§Ã£o ReLU
model.add(Dense(7, input_dim=(len(val_treino.columns)), activation='relu'))
# inicializar a segunda camada com 8 neurÃ´nios e a funÃ§Ã£o ReLU
#model.add(Dense(20, activation='relu'))
#model.add(Dense(20, activation='relu'))
#model.add(Dense(len(val_treino.columns), activation='relu'))
# inicializar a Ãºltima camada (camada de saÃ­da) com um neurÃ´nio e a funÃ§Ã£o Sigmoid
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# treinar o modelo keras

model.fit(val_treino, val_res_treino, epochs=150, batch_size=10,use_multiprocessing=False,verbose=1)


In [ ]:
predictions = model.predict(val_teste)
sum = 0.0
for k in predictions:
  sum += k

md = (sum /len(predictions))-0.05
print(md)
predictionsi = [0 if p < 0.5 else 1 for p in predictions]
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy = accuracy_score(val_res_teste, predictionsi)
mt = confusion_matrix(val_res_teste, predictionsi)
print(mt,f'acuracia de {accuracy} %')
for i,x in enumerate(predictionsi):
  if x != val_res_teste.values[i]:
    print(val_teste.values[i][0:],predictions[i],x,val_res_teste.values[i])
  #print(f'data={val_teste.values[i]} {predictions[i]}% {val_res_teste.values[i]} {x}')
